<div class="alert alert-block alert-info">
25/30:  <br>
As you may imagine, negative numbers shoudn't be there! Please look at the HW solutions.<br>
Graded by DL. 
</div>


# Homework 9.1 : Heavy-tailed distributions and outliers

David Prober’s lab studies sleep using zebrafish as a model organism. In a paper by Gandhi and coworkers his lab studied the effect of a deletion in the gene coding for arylalkylamine N-acetyltransferase (aanat), which is a key enzyme in the rhythmic production of melatonin. Melatonin is a hormone responsible for regulation of circadian rhythms. It is often taken as a drug to treat sleep disorders. The goal of this study was to investigate the effects of aanat deletion on sleep pattern in 5+ day old zebrafish larvae.

In one of the analyses, the authors compared the average activity of multiple fish over the course of night 6 of their lives. Here, activity is defined as the number of seconds per ten minutes that the fish is moving. 

In performing exploratory data analysis, you will see that there are some clear outliers in activity. For at least two of these outliers, domain experts have told me that there were developmental problems with the fish.

In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import numba
import os,sys
import iqplot
#import bebi103

import bokeh.io
bokeh.io.output_notebook()

data_path = "../../data/"

Loading BokehJS ...


a) Assuming that the activity of fish for each respective genotype is Normally distributed, obtain MLEs for the mean activity (parameter ) for each genotype with confidence intervals.


In [4]:
# Load DataFrame
df= pd.read_csv(os.path.join(data_path, 'gandhi_et_al_night_six_activity.csv'), comment='#')
df.head()

# plot each dataponit, and we can see some outliers
p = iqplot.strip(
    data=df,
    q='activity',
    cats=['genotype'],
    jitter=True,
)
bokeh.io.show(p)

In [42]:
# catagrize the data by genotype
df_wt = df[df['genotype']=='wt']
df_mut = df[df['genotype']=='mut']
df_het = df[df['genotype']=='het']

In [43]:
# define a function that calculate the logpdf of normal distribution
def log_like_normal_params(params, n):
    """Log likelihood for i.i.d. normal measurements with input being logarithm of parameters.    """
    mu, sigma = params
   
    return np.sum(scipy.stats.norm.logpdf(n, mu, sigma))

In [44]:
import warnings
# define a function that calculate the parameters and confidential intervals 
def find_mle(n):
    """Perform maximum likelihood estimates for parameters mu, sigma"""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        res = scipy.optimize.minimize(
            fun=lambda params, n : -log_like_normal_params(params, n),
            x0=np.array([15,5]),
            args=(n,),
            method='BFGS'
        )

    if res.success:
        return res.x
    else:
        raise RuntimeError('Convergence failed with message', res.message)

In [49]:
def draw_bs_reps(data, size):
    """bootstrap replicates of parameters of Normal distribution."""
    bs_reps_mu = np.empty(size)
    bs_reps_sigma = np.empty(size)

    for i in range(size):
        
        data_i = np.random.choice(data,size=size,replace=True)
        res = scipy.optimize.minimize(
            fun= lambda params, data: -log_like_normal_params(params, data),
            x0=np.array([10,4]),
            args=(data_i),
            method='BFGS'
            )
        
        bs_reps_mu[i] = res.x[0]
        bs_reps_sigma[i] = res.x[1]
        
    """find confidence intervals"""
    mu_conf_int = np.percentile(bs_reps_mu,[2.5,97.5])
    sigma_conf_int = np.percentile(bs_reps_sigma,[2.5,97.5])
        
    return mu_conf_int

In [50]:
# use bootstrap to calculate the CI 
wt_mu_mle= find_mle ( df_wt['activity'].values )[0]
wt_mu_ci = draw_bs_reps(df_wt['activity'].values, size=1000)

mut_mu_mle= find_mle ( df_mut['activity'].values )[0]
mut_mu_ci = draw_bs_reps(df_mut['activity'].values, size=1000)

het_mu_mle= find_mle ( df_het['activity'].values )[0]
het_mu_ci = draw_bs_reps(df_het['activity'].values, size=1000)

In [51]:
# take a look
print('mle_mu for wt:', wt_mu_mle,'95% CI_mu for wt:', wt_mu_ci)
print('mle_mu for mut:', mut_mu_mle,'95% CI_mu for mut:', mut_mu_ci)
print('mle_mu for het:', het_mu_mle,'95% CI_mu for het:', het_mu_ci)

mle_mu for wt: 10.105588811587662 95% CI_mu for wt: [ 9.85328388 10.37508472]
mle_mu for mut: 30.379042065002423 95% CI_mu for mut: [28.2059848  32.64374355]
mle_mu for het: 11.252449932629267 95% CI_mu for het: [10.88771459 11.60764254]



b) Normal models tend to fail when there are outliers. This is because of their very light tails. If you have an experiment that you suspect may have major deviations, it is often useful to choose a generative model with heavier tails. To that end, model the activity with a Student-t distribution. Obtain MLEs for the parameter  with error bars. Comment on how the inference changes using this model.

In [191]:
# similar to above, but change to student T distribution
def log_like_t_params(params, n):
    """Log likelihood for i.i.d. T measurements with input being logarithm of parameters.    """
    loc, mu, sigma = params
   
    return np.sum(scipy.stats.t.logpdf(n, loc, mu, sigma))

def find_mle_t(n):
    """Perform maximum likelihood estimates for parameters"""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        res = scipy.optimize.minimize(
            fun=lambda params, n : -log_like_t_params(params, n),
            x0=np.array([100, 20, 10]),
            args=(n,),
            method='BFGS',
            tol=5e-3,
        )

    if res.success:
        return res.x
    else:
        raise RuntimeError('Convergence failed with message', res.message)

In [208]:
def draw_bs_reps(data, size):
    """ bootstrap replicates of parameters of T distribution."""
    bs_reps_mu = np.empty(size)
    bs_reps_sigma = np.empty(size)

    for i in range(size):
        
        data_i = np.random.choice(data,size=size,replace=True)
        res = scipy.optimize.minimize(
            fun= lambda params, data: -log_like_t_params(params, data),
            x0=np.array([100, 20, 10]),
            args=(data_i),
            method='BFGS',  tol=5e-7,
            )
        
        bs_reps_mu[i] = res.x[1]
        bs_reps_sigma[i] = res.x[2]
        
    """find confidence intervals"""
    mu_conf_int = np.percentile(bs_reps_mu,[2.5,97.5])
    sigma_conf_int = np.percentile(bs_reps_sigma,[2.5,97.5])
        
    return mu_conf_int

In [ ]:
# use bootstrap to calculate the CI 
wt_mu_mle= find_mle_t ( df_wt['activity'].values )[1]
wt_mu_ci = draw_bs_reps(df_wt['activity'].values, size=1000)

mut_mu_mle= find_mle_t ( df_mut['activity'].values )[1]
mut_mu_ci = draw_bs_reps(df_mut['activity'].values, size=1000)

het_mu_mle= find_mle_t ( df_het['activity'].values )[1]
het_mu_ci = draw_bs_reps(df_het['activity'].values, size=1000)

In [210]:
# take a look at the mle and CI, given T distribution
print('mle_mu for wt:', wt_mu_mle,'95% CI_mu for wt:', wt_mu_ci)
print('mle_mu for mut:', mut_mu_mle,'95% CI_mu for mut:', mut_mu_ci)
print('mle_mu for het:', het_mu_mle,'95% CI_mu for het:', het_mu_ci)

mle_mu for wt: 10.087106926155036 95% CI_mu for wt: [ 9.82949255 10.33967371]
mle_mu for mut: 29.519980083767265 95% CI_mu for mut: [-2728.67899045  1097.73037993]
mle_mu for het: 11.149990789422901 95% CI_mu for het: [-1464.01643554  5523.35407807]


Using t distribution which accounts for the heavy tail, the mle looks more closer to the data we obtained by experiment. The confidenc interval for mut, which has an extreme outlier, looks unreasonable, which is a demonstation that that outlier datapoint results from developmental problem and should be excluded.